In [5]:
import os
from dotenv import load_dotenv
import pandas as pd
from data_loader import DataLoader
from data_cleaner import DataCleaner
from feature_engineer import FeatureEngineer
from rfm_calculator import RFMCalculator
from saver import ModelSaver
from CLV_model import CLVModel

def main():
    load_dotenv()

    
    best_path = os.getenv('BEST_PATH')
    cust_path = os.getenv('CUST_PATH')
    trx_path = os.getenv('TRX_PATH')

    if not all([best_path, cust_path, trx_path]):
        raise ValueError("Path yok")

    #
    loader = DataLoader(best_path, cust_path, trx_path)
    best, cust, trx = loader.load_data()

    
    merged_trx = pd.merge(trx, cust, on='cb_customer_id', how='inner')
    df = pd.merge(merged_trx, best, on='unique_customer_id', how='left')
    df.drop_duplicates(inplace=True)

    
    cleaner = DataCleaner(df)
    df = cleaner.clean_gender_data()
    df = cleaner.convert_date_columns()
    df = cleaner.drop_unnecessary_columns()

    
    engineer = FeatureEngineer(df)
    df = engineer.add_feature_columns()

    
    rfm_calculator = RFMCalculator(df)
    rfm = rfm_calculator.calculate_rfm()
    rfm = rfm_calculator.assign_rfm_scores()

   
    processed_data = engineer.add_additional_features(rfm)

    
    clv_model = CLVModel(processed_data)
    clv_model.run()

    
    saver = ModelSaver()
    saver.save_model(clv_model, 'clv_model')

if __name__ == "__main__":
    main()

C:\Users\eddem\chippin\feature_engineer.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Test RMSE: 0.43362311880544013
Test R^2 Score: 0.8180214935904436
Test Adjusted R^2 Score: 0.8179941386853088
Model saved to models\clv_model.pkl
